In [3]:
# load in libraries
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

Load in data

In [4]:
learned_data_path = '../processed_EEG_data\clean_learned_JoshIrby.csv'
not_learned_data_path = '../processed_EEG_data\clean_notLearned_JoshIrby.csv'

learned_data = pd.read_csv(learned_data_path)
not_learned_data = pd.read_csv(not_learned_data_path)

Model


In [5]:
# checking how much data for each category
print(f"original learned shape: {learned_data.shape}")
print(f"original not_learned shape: {not_learned_data.shape}")

# make them equal to remove bias
learned_data = learned_data.sample(frac=1, random_state=42).reset_index(drop=True)
not_learned_data = not_learned_data.sample(frac=1, random_state=42).reset_index(drop=True)

# get length 
learned_length = learned_data.shape[0]
not_learned_length = not_learned_data.shape[0]

new_learned_data = learned_data.drop(index=range(not_learned_length, learned_length))

print(f"new learned shape: {new_learned_data.shape}")
print(f"learned shape: {learned_data.shape}")
print(f"new not_learned shape: {not_learned_data.shape}")

original learned shape: (959, 1232)
original not_learned shape: (222, 1232)
new learned shape: (222, 1232)
learned shape: (959, 1232)
new not_learned shape: (222, 1232)


In [7]:
# checking if last line had an impact
print(f"new learned shape: {new_learned_data.shape}")
print(f"new not_learned shape: {not_learned_data.shape}")

# convert pd to a np array for the feature matrix X
X_learned = new_learned_data.values  
X_not_learned = not_learned_data.values

# make the label vector y
'''
to indicate which epochs go to which condition
y vector label should be as long as x matrix is long

zeros = learned
ones = not learned
'''
y_learned = np.zeros(X_learned.shape[0])
y_not_learned = np.ones(X_not_learned.shape[0])

# combine the data and labels
# learned + not learned 
# learned labels + not learned labels
X = np.concatenate([X_learned, X_not_learned], axis=0)
y = np.concatenate([y_learned, y_not_learned], axis=0)

# split into training and test data
test_size_n = 0.2   # sets this percentage aside for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_n, random_state=0)

model = SVC(kernel='rbf')
#from SVM scikit-learn library, uses radial basis function
# use balanced to account for the bias in data, class_weight='balanced'
# without this it counts EVERYTHING as positive

# kernel options: linear, rbf, poly, 


# train the data using fit
model.fit(X_train, y_train)

# make predictions based on the testing data from before
y_pred = model.predict(X_test)

# evauluate the performance of the SVM model by caclulating confusino matrix
# and the accuracy score
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: \n", cm)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

new learned shape: (222, 1232)
new not_learned shape: (222, 1232)
Confusion Matrix: 
 [[36  7]
 [22 24]]
Accuracy:  0.6741573033707865
